In [ ]:
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.model_selection import backtesting_forecaster
from skforecast.model_selection import grid_search_forecaster
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from skforecast.ForecasterAutoreg import ForecasterAutoreg

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

import torch
import os
import sys
import psutil
import warnings
warnings.filterwarnings('ignore')


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')

print(f"CPU RAM Free: {psutil.virtual_memory().available / 1024**3:.2f} GB")

In [3]:
holiday_events_df = pd.read_csv('RNNs/Tensorflow-Keras/Predictions/Store Sales - Time Series Forecasting/holidays_events.csv')
stores_df = pd.read_csv('RNNs/Tensorflow-Keras/Predictions/Store Sales - Time Series Forecasting/stores.csv')
transactions_df = pd.read_csv('RNNs/Tensorflow-Keras/Predictions/Store Sales - Time Series Forecasting/transactions.csv')
train_df = pd.read_csv('RNNs/Tensorflow-Keras/Predictions/Store Sales - Time Series Forecasting/train.csv')
test_df = pd.read_csv('RNNs/Tensorflow-Keras/Predictions/Store Sales - Time Series Forecasting/test.csv')
oil_df = pd.read_csv('RNNs/Tensorflow-Keras/Predictions/Store Sales - Time Series Forecasting/oil.csv')

In [ ]:
holiday_events_df = holiday_events_df.sort_values(by='transferred', ascending=False).drop_duplicates(subset='date', keep='first')
holiday_events_df[holiday_events_df['transferred'] == True].head()
holiday_events_df.head()

In [ ]:
stores_df.head()

In [ ]:
transactions_df.head()

In [ ]:
train_df.head()

In [ ]:
oil_df['dcoilwtico'] = oil_df['dcoilwtico'].bfill()
oil_df.head()

In [ ]:
full_train_df = train_df.merge(stores_df, how='left', on='store_nbr').merge(transactions_df, how='left', on=['date', 'store_nbr']).merge(oil_df, how='left', on='date').merge(holiday_events_df, how='left', on='date')
full_train_df.head()

In [ ]:
full_test_df = test_df.merge(stores_df, how='left', on='store_nbr').merge(transactions_df, how='left', on=['date', 'store_nbr']).merge(oil_df, how='left', on='date').merge(holiday_events_df, how='left', on='date')
full_test_df.head()

In [11]:
# Re-format the date column and set it as index
full_train_df['date'] = pd.to_datetime(full_train_df['date'], format = '%Y-%m-%d')
# Re-format the date column and set it as index
full_test_df['date'] = pd.to_datetime(full_test_df['date'], format = '%Y-%m-%d')

In [12]:
full_train_df['day_of_week'] = full_train_df['date'].dt.dayofweek
full_test_df['day_of_week'] = full_test_df['date'].dt.dayofweek

In [13]:
full_train_df['month'] = full_train_df['date'].dt.month
full_test_df['month'] = full_test_df['date'].dt.month

In [14]:
# Reset the index to access 'date' column
full_train_df.reset_index(inplace=True)
full_test_df.reset_index(inplace=True)

# Set the date as index
full_train_df.set_index('date', inplace=True)
full_test_df.set_index('date', inplace=True)

In [ ]:
# Set the dataset frequency to be (D)aily data
# full_train_df = full_train_df.asfreq('D', method = 'bfill') 
# full_test_df = full_test_df.asfreq('D', method = 'bfill') 
# Fill missing value with the latest available data

full_train_df.head()

In [16]:
# Encoding the type of products
family_label_encoder = LabelEncoder()
full_train_df['family_encoded'] = family_label_encoder.fit_transform(full_train_df['family'])
full_test_df['family_encoded'] = family_label_encoder.transform(full_test_df['family'])

#Encoding the citys and states
city_label_encoder = LabelEncoder()
full_train_df['city_encoded'] = city_label_encoder.fit_transform(full_train_df['city'])
full_test_df['city_encoded'] = city_label_encoder.transform(full_test_df['city'])

state_label_encoder = LabelEncoder()
full_train_df['state_encoded'] = state_label_encoder.fit_transform(full_train_df['state'])
full_test_df['state_encoded'] = state_label_encoder.transform(full_test_df['state'])

#Encoding the type of stores
type_store_label_encoder = LabelEncoder()
full_train_df['type_store_encoded'] = state_label_encoder.fit_transform(full_train_df['type_x'])
full_test_df['type_store_encoded'] = state_label_encoder.transform(full_test_df['type_x'])

#Transactions fillna
full_train_df['transactions'] = full_train_df['transactions'].fillna(0)
full_test_df['transactions'] = full_test_df['transactions'].fillna(0)
#Transactions Scale
transactions_scaler = MinMaxScaler()
full_train_df['transactions_scaled'] = transactions_scaler.fit_transform(full_train_df[['transactions']])
full_test_df['transactions_scaled'] = transactions_scaler.transform(full_test_df[['transactions']])

#Encoding the type of holidays
type_holiday_label_encoder = LabelEncoder()
full_train_df['type_y'].fillna('no_holiday', inplace=True)
full_test_df['type_y'].fillna('no_holiday', inplace=True)
combined_type_y = pd.concat([full_train_df['type_y'], full_test_df['type_y']])
type_holiday_label_encoder.fit(combined_type_y)
full_train_df['type_holiday_encoded'] = type_holiday_label_encoder.transform(full_train_df['type_y'])
full_test_df['type_holiday_encoded'] = type_holiday_label_encoder.transform(full_test_df['type_y'])

#Encoding the type of holidays
locale_label_encoder = LabelEncoder()
full_train_df['locale'].fillna('no_holiday', inplace=True)
full_test_df['locale'].fillna('no_holiday', inplace=True)
combined_type_y = pd.concat([full_train_df['locale'], full_test_df['locale']])
locale_label_encoder.fit(combined_type_y)
full_train_df['locale_encoded'] = locale_label_encoder.transform(full_train_df['locale'])
full_test_df['locale_encoded'] = locale_label_encoder.transform(full_test_df['locale'])


#Encoding the type of holidays
transferred_label_encoder = LabelEncoder()
full_train_df['transferred'].fillna('no_holiday', inplace=True)
full_test_df['transferred'].fillna('no_holiday', inplace=True)
full_train_df['transferred'].replace({True: 'transferred', False: 'not_transferred'}, inplace=True)
full_test_df['transferred'].replace({True: 'transferred', False: 'not_transferred'}, inplace=True)
combined_type_y = pd.concat([full_train_df['transferred'], full_test_df['transferred']])
transferred_label_encoder.fit(combined_type_y)
full_train_df['transferred_encoded'] = transferred_label_encoder.transform(full_train_df['transferred'])
full_test_df['transferred_encoded'] = transferred_label_encoder.transform(full_test_df['transferred'])

oil_scaler = MinMaxScaler()
full_train_df['dcoilwtico'] = full_train_df['dcoilwtico'].bfill()
full_test_df['dcoilwtico'] = full_test_df['dcoilwtico'].bfill()
full_train_df['dcoilwtico_scaled'] = oil_scaler.fit_transform(full_train_df[['dcoilwtico']])
full_test_df['dcoilwtico_scaled'] = oil_scaler.transform(full_test_df[['dcoilwtico']])

sales_scaler = MinMaxScaler()
full_train_df['sales_scaled'] = sales_scaler.fit_transform(full_train_df[['sales']])
# full_test_df['sales_scaled'] = sales_scaler.transform(full_test_df[['sales']])



In [ ]:
train_data=full_train_df[['store_nbr', 'onpromotion', 'cluster', 'day_of_week', 'month', 'family_encoded', 'city_encoded', 'state_encoded', 'type_store_encoded', 'type_holiday_encoded', 'locale_encoded', 'transferred_encoded', 'dcoilwtico', 'sales_scaled']]
test_data=full_test_df[['store_nbr', 'onpromotion', 'cluster', 'day_of_week', 'month', 'family_encoded', 'city_encoded', 'state_encoded', 'type_store_encoded', 'type_holiday_encoded', 'locale_encoded', 'transferred_encoded', 'dcoilwtico']]

# Ensure the date ranges are correctly defined
train_data_start=pd.to_datetime('2013-01-01')
train_data_end=pd.to_datetime('2017-08-15')

test_data_start=pd.to_datetime('2017-08-16')
test_data_end=pd.to_datetime('2017-08-31')

train_data.head()



In [18]:
# Group the train_data by 'store_nbr' and 'family'
grouped_train = train_data.groupby(['store_nbr', 'family_encoded'])
# Group the train_data by 'store_nbr' and 'family'
grouped_test = test_data.groupby(['store_nbr', 'family_encoded'])

# Create an array of DataFrames
dataframes_array_train = [group for _, group in grouped_train]

# Create an array of DataFrames
dataframes_array_test = [group for _, group in grouped_test]




In [19]:
# Add a row with the day 2012-12-25 to each DataFrame in dataframes_array_train
for idx, df in enumerate(dataframes_array_train):
    for i in range(2013, 2017):
        new_row = pd.DataFrame({
            'date': [pd.to_datetime(f'{i}-12-25')],
            'store_nbr': [df['store_nbr'].iloc[0]],
            'onpromotion': [0],
            'cluster': [df['cluster'].iloc[0]],
            'day_of_week': [df.loc[f'{i}-12-24', 'day_of_week'] + 1 if df.loc[f'{i}-12-24', 'day_of_week'] + 1 != 8 else 1],  # Assuming 1 for Monday, adjust as needed
            'month': [12],
            'family_encoded': [df['family_encoded'].iloc[0]],
            'city_encoded': [df['city_encoded'].iloc[0]],
            'state_encoded': [df['state_encoded'].iloc[0]],
            'type_store_encoded': [df['type_store_encoded'].iloc[0]],
            'type_holiday_encoded': [1],  # Assuming 1 for holiday, adjust as needed
            'locale_encoded': [1],  # Assuming 1 for locale, adjust as needed
            'transferred_encoded': [0],  # Assuming 0 for not transferred, adjust as needed
            'dcoilwtico': [df.loc[f'{i}-12-24', 'dcoilwtico']],  # Assuming a default value, adjust as needed
            'sales_scaled': [0]  # Assuming no sales, adjust as needed
        })
        new_row.set_index('date', inplace=True)
        df = pd.concat([df, new_row])
        df.sort_index(inplace=True)
        #df.reset_index(inplace=True)
        #df.set_index('date', inplace=True)
    dataframes_array_train[idx] = df



In [ ]:
dataframes_array_train[0].head()

In [ ]:
# Generate a complete date range
complete_date_range = pd.date_range(start=train_data_start, end=train_data_end)

# Find missing dates
missing_dates = complete_date_range.difference(dataframes_array_train[0].loc[train_data_start:train_data_end].index)

# Display missing dates
print(missing_dates)

In [ ]:
# Keep only unique rows in each DataFrame in dataframes_array_train
dataframes_array_train = [df.drop_duplicates(keep='first') for df in dataframes_array_train]

# Fill missing dates in each DataFrame in dataframes_array_train
for df in dataframes_array_train:
#    complete_date_range = pd.date_range(start=df.index.min(), end=df.index.max())
#    df = df.reindex(complete_date_range, method='ffill')
#    df.sort_index(inplace=True)
    df.asfreq('D', method='bfill')

# Display the first few DataFrames in the array
for df in dataframes_array_train[:3]:
    print(df.head())

for df in dataframes_array_test:
#    complete_date_range = pd.date_range(start=df.index.min(), end=df.index.max())
#    df = df.reindex(complete_date_range, method='ffill')
#    df.sort_index(inplace=True)
    df.asfreq('D', method='bfill')

# Display the first few DataFrames in the array
for df in dataframes_array_train[:3]:
    print(df.head())

# Create an array of DataFrames
dataframes_array_test = [group for _, group in grouped_test]
for df in dataframes_array_test[:3]:
    print(df.head())
    



In [ ]:
target = {}
for idx, df in enumerate(dataframes_array_train):
    target[idx] = df.loc[train_data_start:train_data_end, df.columns == 'sales_scaled']
    
target[0].head()


In [24]:
# Complete target with 0 between test_data_start and test_data_end
for idx, df in target.items():
    complete_date_range = pd.date_range(start=test_data_start, end=test_data_end)
    missing_dates = complete_date_range.difference(df.index)
    for date in missing_dates:
        df.loc[date] = 0
    df.sort_index(inplace=True)
    df.asfreq('D', method='bfill')


In [25]:
for df in dataframes_array_train:
    df.drop(columns=['sales_scaled'], inplace=True)

In [26]:
for c in range(0, len(dataframes_array_train)):
    dataframes_array_train[c] = pd.concat([dataframes_array_train[c], dataframes_array_test[c]])
    

In [ ]:
# Generate a complete date range
complete_date_range = pd.date_range(start=train_data_start, end=train_data_end)

# Find missing dates
missing_dates = complete_date_range.difference(target[0].loc[train_data_start:train_data_end].index)

# Display missing dates
missing_dates


In [ ]:
print(target[0].loc[train_data_start:test_data_end, 'sales_scaled'].head())
print(target[0].loc[train_data_start:test_data_end, 'sales_scaled'].tail())

In [ ]:

for df in target.values():
    df.index = pd.to_datetime(df.index)
    df.asfreq('D', method='bfill')
    
target[0].head()


In [30]:
exog = dataframes_array_train[0].loc[train_data_start:train_data_end, ['store_nbr', 'onpromotion', 'cluster', 'day_of_week', 'month', 'family_encoded', 'city_encoded', 'state_encoded', 'type_store_encoded', 'type_holiday_encoded', 'locale_encoded', 'transferred_encoded', 'dcoilwtico']]
exog = exog.asfreq('D')
target_test=target[0].loc[train_data_start:train_data_end, 'sales_scaled']
target_test=target_test.asfreq('D')


In [ ]:

target_test=target[1].loc[train_data_start:train_data_end, 'sales_scaled']
target_test=target_test.asfreq('D')
target_test.head()


In [33]:


final_df = pd.DataFrame()

for i in range(0, len(dataframes_array_train)):
    # Define the forecaster with multiple series
    forecaster = ForecasterAutoreg(
        #regressor = XGBRegressor(
        #                          n_estimators=1000,
        #                          tree_method='hist',
        #                          device='cuda'
        #                        ),
        regressor = LGBMRegressor(random_state = 432,
                                force_col_wise=True,
                                verbose=-1,
                                max_depth=-1,
                                ),
        lags = 30,
    )


    # Parameter Grid for Regressor
    param_grid = {
        # max_depth: The maximum depth of the tree. If None, then nodes are expanded until all leaves are pure or until all leaves contain less than min_samples_split samples.
        #'max_depth' : [-1, 1, 3, 5, 10],
        # min_samples_split: The minimum number of samples required to split an internal node.
        #'min_samples_split' : [2, 3, 4, 5, 6, 10],
        # ccp_alpha: Complexity parameter used for Minimal Cost-Complexity Pruning. The subtree with the largest cost complexity that is smaller than ccp_alpha will be chosen.
        #'ccp_alpha' : [0.0, 0.001, 0.01]
        'learning_rate': [0.01, 0.1],
    }

    # lags_grid: The number of previous observations to consider
    lags_grid = [30, 50, 100, 150, [1,2,3,4,5,7,9,13,15,20,30,31,29,28,50], 10, 20]


    exog = dataframes_array_train[i].loc[train_data_start:train_data_end, ['store_nbr', 
                                                                           #'onpromotion', 
                                                                           #'cluster', 
                                                                           'day_of_week', 
                                                                           'month', 
                                                                           'family_encoded', 
                                                                           #'city_encoded', 
                                                                           #'state_encoded', 
                                                                           'type_store_encoded', 
                                                                           'type_holiday_encoded', 
                                                                           'locale_encoded', 
                                                                           'transferred_encoded', 
                                                                           'dcoilwtico']]
    exog = exog.asfreq('D').ffill()
    target_test=target[i].loc[train_data_start:train_data_end, 'sales_scaled']
    target_test=target_test.asfreq('D').ffill()  # Fill missing values



    # Grid Search with Refit and Increasing Train Size
    grid_forecaster = grid_search_forecaster(
        # forecaster - our model
        forecaster = forecaster,
        # y - target variable
        y = target_test,
        # exog - exogenous variables
        exog = exog,
        # param_grid - dictionary with parameters to test
        param_grid = param_grid,
        # lag_grid - list with lags to test
        lags_grid = lags_grid,
        # steps - number of steps to forecast
        steps = len(dataframes_array_train[i].loc[test_data_start:test_data_end]),
        # refit - refit the best model with all data
        refit = True,
        # metric - metric to optimize
        metric = 'mean_squared_error',
        # initial_train_size - initial size of the train data
        initial_train_size = len(dataframes_array_train[i].loc[train_data_start:train_data_end])-1,
        # fixed_train_size - if True, the training size is fixed
        fixed_train_size = False,
        # return_best - return the best model
        return_best = True,
        # verbose - show all messages
        verbose = False
    )

    # Backtest RMSE
    np.sqrt(grid_forecaster.iloc[0]['mean_squared_error'])
    
    
    forecaster.fit(y=target_test, exog=exog)
    # prediction
    exog = dataframes_array_train[i].loc[test_data_start:test_data_end, ['store_nbr', 
                                                                         #'onpromotion', 
                                                                         #'cluster', 
                                                                         'day_of_week', 
                                                                         'month', 
                                                                         'family_encoded', 
                                                                         #'city_encoded', 
                                                                         #'state_encoded', 
                                                                         'type_store_encoded', 
                                                                         'type_holiday_encoded', 
                                                                         'locale_encoded', 
                                                                         'transferred_encoded', 
                                                                         'dcoilwtico']]
    exog.index = pd.to_datetime(exog.index)
    exog = exog.asfreq('D').ffill()
    predicted_values = forecaster.predict(steps=len(dataframes_array_train[i].loc[test_data_start:test_data_end]), exog=exog)
    predicted_values = pd.DataFrame({'store_nbr': [dataframes_array_train[i]['store_nbr'].iloc[0]], 'family_encoded': [dataframes_array_train[i]['family_encoded'].iloc[0]], 'sales_scaled': predicted_values}, index=exog.index)
    final_df = pd.concat([final_df, predicted_values])
    print (str(i) + ' of ' + str(len(dataframes_array_train)))
    
# Fit the model using train data with multiple series
#exog = dataframes_array_train[0].loc[train_data_start:train_data_end, ['store_nbr', 'onpromotion', 'cluster', 'day_of_week', 'month', 'family_encoded', 'city_encoded', 'state_encoded', 'type_store_encoded', 'type_holiday_encoded', 'locale_encoded', 'transferred_encoded', 'dcoilwtico']]
#exog = exog.asfreq('D')
#target_test=target[0].loc[train_data_start:train_data_end, 'sales']
#target_test=target_test.asfreq('D')
#forecaster.fit(y=target_test, exog=exog)




lags grid: 100%|██████████████████████████████████████████████████████████████████████████| 7/7 [00:01<00:00,  4.16it/s]


`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50] 
  Parameters: {'learning_rate': 0.01}
  Backtesting metric: 1.606906208602189e-11

1593 of 1782
Number of models compared: 14.


lags grid: 100%|██████████████████████████████████████████████████████████████████████████| 7/7 [00:01<00:00,  4.32it/s]


`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30] 
  Parameters: {'learning_rate': 0.1}
  Backtesting metric: 2.5143873878301586e-09

1594 of 1782
Number of models compared: 14.


lags grid: 100%|██████████████████████████████████████████████████████████████████████████| 7/7 [00:01<00:00,  4.36it/s]


...

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50] 
  Parameters: {'learning_rate': 0.1}
  Backtesting metric: 8.700986970217817e-08

1779 of 1782
Number of models compared: 14.


lags grid: 100%|██████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00,  7.77it/s]


`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100] 
  Parameters: {'learning_rate': 0.1}
  Backtesting metric: 1.3227397495821534e-13

1780 of 1782
Number of models compared: 14.


lags grid: 100%|██████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00,  7.43it/s]


`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  7  9 13 15 20 30 31 29 28 50] 
  Parameters: {'learning_rate': 0.1}
  Backtesting metric: 3.932548991508714e-11

1781 of 1782


In [34]:
final_df.tail()

,store_nbr,family_encoded,sales_scaled
date,,,
2017-08-27,54,32,0.000020
2017-08-28,54,32,0.000022
2017-08-29,54,32,0.000026
2017-08-30,54,32,0.000006
2017-08-31,54,32,0.000021


In [35]:
final_df['sales'] = sales_scaler.inverse_transform(final_df[['sales_scaled']])

In [36]:
# Ensure the date column is in the index for both dataframes
final_df.reset_index(inplace=True)
full_test_df.reset_index(inplace=True)

# Merge the dataframes on 'family_encoded', 'store_nbr', and 'date'
merged_df = pd.merge(final_df, full_test_df, on=['family_encoded', 'store_nbr', 'date'], how='inner')

# Display the merged dataframe
merged_df.head()

,date,store_nbr,family_encoded,sales_scaled,sales,index,id,family,onpromotion,city,...,day_of_week,month,city_encoded,state_encoded,type_store_encoded,transactions_scaled,type_holiday_encoded,locale_encoded,transferred_encoded,dcoilwtico_scaled
0,2017-08-16,1,0,0.000033,4.160138,0,3000888,AUTOMOTIVE,0,Quito,...,2,8,18,12,3,0.0,6,3,0,0.244108
1,2017-08-17,1,0,0.000028,3.526624,1782,3002670,AUTOMOTIVE,0,Quito,...,3,8,18,12,3,0.0,6,3,0,0.247305
2,2017-08-18,1,0,0.000030,3.734434,3564,3004452,AUTOMOTIVE,0,Quito,...,4,8,18,12,3,0.0,6,3,0,0.265309
3,2017-08-19,1,0,0.000036,4.539346,5346,3006234,AUTOMOTIVE,0,Quito,...,5,8,18,12,3,0.0,6,3,0,0.251096
4,2017-08-20,1,0,0.000017,2.156580,7128,3008016,AUTOMOTIVE,0,Quito,...,6,8,18,12,3,0.0,6,3,0,0.251096


In [37]:
submission_df=merged_df[['id', 'sales']]
submission_df.head()

,id,sales
0,3000888,4.160138
1,3002670,3.526624
2,3004452,3.734434
3,3006234,4.539346
4,3008016,2.156580


In [ ]:
!pwd

In [38]:
submission_df.to_csv('RNNs/Tensorflow-Keras/Predictions/Store Sales - Time Series Forecasting/submission_6.csv', index=False)